In [121]:
# Condensing (Smoothing Prep)

In [122]:
# Imports

import pandas as pd
import numpy as np

In [123]:
p_path = 'p18'
file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master/'+p_path+'_master.csv'
data = pd.read_csv(file)

/anaconda3/envs/caispp/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (49,68) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [124]:
data = data.sort_values(['session_num', 'timestamp'], ascending=[True, True])

In [125]:
orig_len = len(data)

In [126]:
data.head()

,session_num,timestamp,engagement,participant,of_AU01_c,of_AU02_c,of_AU04_c,of_AU05_c,of_AU06_c,of_AU07_c,...,ros_mistake_made,ros_mistakes_game,ros_mistakes_session,ros_skill,ros_skill_EM,ros_skill_NC,ros_skill_OS,ros_ts_attempt,ros_ts_game_start,ros_ts_robot_talked
0,3.0,0.000,0,18,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.00000
1,3.0,0.033,0,18,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.03333
2,3.0,0.067,0,18,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.06667
3,3.0,0.100,0,18,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.10000
4,3.0,0.133,0,18,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.13333


In [127]:
data.columns

Index(['session_num', 'timestamp', 'engagement', 'participant', 'of_AU01_c',
       'of_AU02_c', 'of_AU04_c', 'of_AU05_c', 'of_AU06_c', 'of_AU07_c',
       'of_AU09_c', 'of_AU10_c', 'of_AU12_c', 'of_AU14_c', 'of_AU15_c',
       'of_AU17_c', 'of_AU20_c', 'of_AU23_c', 'of_AU25_c', 'of_AU26_c',
       'of_AU28_c', 'of_AU45_c', 'of_confidence', 'of_gaze_0_x', 'of_gaze_0_y',
       'of_gaze_0_z', 'of_gaze_1_x', 'of_gaze_1_y', 'of_gaze_1_z',
       'of_gaze_angle_x', 'of_gaze_angle_y', 'of_gaze_distance',
       'of_gaze_distance_x', 'of_gaze_distance_y', 'of_pose_Rx', 'of_pose_Ry',
       'of_pose_Rz', 'of_pose_Tx', 'of_pose_Ty', 'of_pose_Tz',
       'of_pose_distance', 'of_success', 'op_num_people', 'a_harmonicity',
       'a_intensity', 'a_mfcc_0', 'a_mfcc_1', 'a_pitch_frequency',
       'a_pitch_strength', 'ros_GAME_STATE', 'ros_PARTICIPANT_STATE',
       'ros_ROBOT_STATE', 'ros_activity', 'ros_aptitude', 'ros_diff_1',
       'ros_diff_2', 'ros_diff_3', 'ros_diff_4', 'ros_diff_5',
      

# 1. Remove Beginning of Videos/Sessions

In [128]:
len(data.loc[data['engagement']==1])/len(data)

0.6607581297334003

In [129]:
# Remove data @ beginning of each session before first game has been played 

last_session = -1
session_start = -1 
track = False

# tuples: (session_start, first_game_start)
windows_to_remove = [] 

for i,r in data.iterrows():
    # Check if new session started
    if last_session != r['session_num']:
        session_start = i
        last_session = r['session_num']
        track = True 
    
    if track:
        # check if we've found our first game start! 
        if r['ros_game_start'] == 1: 
            windows_to_remove.append((session_start, i))
            track = False 

In [130]:
for i in windows_to_remove:
    print(i)

(0, 840)
(39617, 39947)
(63520, 63790)
(94720, 95350)
(114166, 114706)
(137349, 137979)
(179062, 179542)
(210359, 210809)
(227698, 228058)
(248633, 249083)
(267236, 268316)
(292573, 293053)
(306660, 307110)
(320592, 320982)
(334563, 335013)


In [131]:
# Remove data at beginning
# Iterate backwards! 

n = len(windows_to_remove)
for i in range(n-1, -1, -1):
    rem = windows_to_remove[i]

    if (rem[0] == rem[1]):
        continue

    seg1 = data.loc[:rem[0]-1]
    seg2 = data.loc[rem[1]:, :]
    data = seg1.append(seg2, sort=True)
    
data = data.reset_index(drop=True)

In [132]:
curr_len = len(data)
print((orig_len-curr_len)/orig_len)

0.021971114858703


In [133]:
for i in data['session_num'].unique():
    temp = data[data['session_num']==i]
    print(i, min(temp['timestamp']))

3.0 28.0
4.0 11.0
5.0 9.0
7.0 21.0
8.0 18.0
9.0 21.0
10.0 16.0
11.0 15.0
12.0 12.0
13.0 15.0
15.0 36.0
17.0 16.0
18.0 15.0
19.0 13.0
20.0 15.0


# 2. Remove Ending of Videos/Sessions

In [134]:
# Remove data @ end of each session after last game has been played 

# Iterate Backwards! 

n = len(data)

last_session = -1
session_end = -1 
track = False

# tuples: (last_game_end, session_end)
windows_to_remove = [] 

for i in range(n-1, -1, -1):
    sess = data.loc[i, 'session_num']
    if last_session != sess:
        session_end = i
        last_session = sess
        track = True
    
    if track:
        # check if we've found end of last game! 
        if data.loc[i, 'ros_game_correct'] == 1 or data.loc[i, 'ros_game_incorrect'] == 1 or data.loc[i, 'ros_mistake_made'] == 1:
            windows_to_remove.append((i, session_end))
            track = False

In [135]:
windows_to_remove

[(344253, 348546),
 (323052, 327182),
 (309450, 313601),
 (295393, 300119),
 (282206, 286512),
 (256853, 262255),
 (240328, 244102),
 (219449, 223527),
 (201772, 206638),
 (171999, 175821),
 (130396, 134738),
 (107590, 112095),
 (89380, 93279),
 (58097, 62349),
 (34200, 38776)]

In [136]:
# Remove data at ending
# windows to remove has end sessions first, which is what we want to remove first

for rem in windows_to_remove:
    if (rem[0] == rem[1]):
        continue

    seg1 = data.loc[:rem[0]]
    seg2 = data.loc[rem[1]+1:, :]
    data = seg1.append(seg2, sort=True)
    
data = data.reset_index(drop=True)

In [137]:
curr_len = len(data)
print((orig_len-curr_len)/orig_len)

0.2047045684766413


In [138]:
curr_len/orig_len

0.7952954315233587

In [139]:
for i in data['session_num'].unique():
    temp = data[data['session_num']==i]
    print(i, int(max(temp['timestamp'])/60), int(max(temp['timestamp'])%60))

3.0 19 28
4.0 10 55
5.0 15 10
7.0 8 18
8.0 10 28
9.0 21 3
10.0 14 41
11.0 7 22
12.0 9 32
13.0 7 20
15.0 11 41
17.0 5 12
18.0 5 26
19.0 5 28
20.0 9 44


In [140]:
# save pre-smoothing data

file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master2/'+p_path+'_master.csv'
data.to_csv(file, index=False)